In [1]:
import cv2
import numpy as np
import pytesseract
import os
import re
import pandas as pd

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
import matplotlib.pylab as plt
import tensorflow as tf

folder = r"../../../1.DATA/3.forms_images/Registry2018/"
year = '2018'
forms_list = os.listdir(folder)

# print(forms_list)
#  Load the model
from tensorflow.keras.models import load_model
model = load_model('checkboxes_classifier.h5')

roi = [[(115,143),(127,154), 'box', 'Activity.Industry'],
        [(202,143),(213,154), 'box', 'Activity.Commerce'],
        [(280,143),(291,154), 'box', 'Activity.Gas_Station'], 
        [(414,143),(426,154), 'box', 'Activity.Residue'],
        [(493,143),(505,154), 'box', 'Activity.Accidents'],
        [(588,143),(599,154), 'box', 'Activity.Agriculture'],
        [(667,145),(679,157), 'box', 'Activity.Unknown'],
        [(618,181),(631,194), 'box', 'Reuse'], 
       
        [(55,223),(68,235), 'box', 'ManagementPhase.OccurenceAssessment'], 
        [(55,245),(68,258), 'box', 'ManagementPhase.LeakRemovalMeasures'],
        [(55,270),(68,282), 'box', 'ManagementPhase.ConfirmatoryInvestigation'],
        [(55,294),(68,306), 'box', 'ManagementPhase.DetailedInvestigation&InterventionPlan'],
        [(55,312),(68,325), 'box', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring'],
        [(55,337),(68,349), 'box', 'ManagementPhase.ClosingMonitoring'], 
        [(390,223),(402,235), 'box', 'ManagementPhase.PreliminaryAssessment'],
        [(390,241),(402,254), 'box', 'ManagementPhase.ConfirmatoryInvestigation1'],
        [(390,265),(402,277), 'box', 'ManagementPhase.DetailedInvestigation'],
        [(390,282),(402,294), 'box', 'ManagementPhase.RiskAssessment'],
        [(390,300),(402,313), 'box', 'ManagementPhase.InterventionPlan'], 
        [(390,321),(402,333), 'box', 'ManagementPhase.RemediationProject'],
        [(390,341),(402,353), 'box', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring1'],
        [(390,359),(402,372), 'box', 'ManagementPhase.ClosingMonitoring1'],


        [(39,407),(52,419), 'box', 'ContaminationSource.Storage'],
        [(39,425),(52,438), 'box', 'ContaminationSource.Disposal'],
        [(173,407),(186,419), 'box', 'ContaminationSource.Production'],
        [(175,425),(187,438), 'box', 'ContaminationSource.Infiltration'],
        [(289,407),(301,419), 'box', 'ContaminationSource.Maintenance'],
        [(287,428),(300,441), 'box', 'ContaminationSource.Accidents'],
        [(415,407),(428,419), 'box', 'ContaminationSource.AtmosphericEmissions'],
        [(416,425),(429,438), 'box', 'ContaminationSource.Unknown'],
        [(588,407),(600,419), 'box', 'ContaminationSource.WastewaterTreatment'], 
       
        [(206,512),(218,526), 'box', 'ImpactedMedia.SuperficialSoil.Inside'],
        [(287,512),(300,526), 'box', 'ImpactedMedia.SuperficialSoil.Outside'],
        [(206,533),(218,545), 'box', 'ImpactedMedia.Soil.Inside'], 
        [(287,533),(300,545), 'box', 'ImpactedMedia.Soil.Outside'],
        [(206,548),(218,561), 'box', 'ImpactedMedia.SurfaceWater.Inside'],
        [(287,548),(300,561), 'box', 'ImpactedMedia.SurfaceWater.Outside'],
        [(206,566),(218,579), 'box', 'ImpactedMedia.Groundwater.Inside'],
        [(287,566),(300,579), 'box', 'ImpactedMedia.Groundwater.Outside'],
        [(206,586),(218,598), 'box', 'ImpactedMedia.Sediments.Inside'], 
        [(287,586),(300,598), 'box', 'ImpactedMedia.Sediments.Outside'],
        [(207,603),(219,615), 'box', 'ImpactedMedia.Air.Inside'],
        [(289,603),(301,615), 'box', 'ImpactedMedia.Air.Outside'],
        [(207,620),(219,633), 'box', 'ImpactedMedia.Biota.Inside'],
        [(289,620),(301,633), 'box', 'ImpactedMedia.Biota.Outside'],
 
        [(394,468),(406,481), 'box', 'Contaminants.Metals'], 
        [(394,493),(406,505), 'box', 'Contaminants.OtherInorganic'],
        [(394,517),(406,529), 'box', 'Contaminants.HalogenatedSolvents'],
        [(394,539),(406,552), 'box', 'Contaminants.AromaticSolvents'],
        [(394,564),(406,576), 'box', 'Contaminants.HalogenatedAromaticSolvents'], 
        [(394,586),(406,599), 'box', 'Contaminants.PAHs'], 
        [(394,611),(406,623), 'box', 'Contaminants.PCBs'], 
        [(394,628),(406,640), 'box', 'Contaminants.Methane'],
        [(394,648),(406,660), 'box', 'Contaminants.AutomotiveFuels'],
        [(394,667),(406,680), 'box', 'Contaminants.OtherVapours/Gazes'],
        [(645,469),(657,482), 'box', 'Contaminants.Fenols'],
        [(645,494),(657,506), 'box', 'Contaminants.Biocides'], 
        [(645,518),(657,530), 'box', 'Contaminants.Phthalates'], 
        [(645,540),(657,553), 'box', 'Contaminants.Dioxins&Furans'],
        [(645,565),(657,577), 'box', 'Contaminants.Anilines'],
        [(645,587),(657,600), 'box', 'Contaminants.Radionuclides'],
        [(645,612),(657,624), 'box', 'Contaminants.Microbiological'], 
        [(646,636),(658,648), 'box', 'Contaminants.TPH'], 
        [(645,656),(657,668), 'box', 'Contaminants.Others'],
        [(44,656),(56,669), 'box', 'Contaminants.FreePhase'],
        [(44,675),(56,687), 'box', 'Contaminants.POPs'], 

        [(60,714),(73,727), 'box', 'EmergencyMeasures.IsolationOfTheArea'],
        [(60,731),(73,744), 'box', 'EmergencyMeasures.Ventilation/ExhaustionOfConfinedSpaces'],
        [(60,749),(73,761), 'box', 'EmergencyMeasures.MonitoringTheExplosivenessIndex'], 
        [(60,764),(73,777), 'box', 'EmergencyMeasures.EnvironmentalMonitoring'], 
        [(60,781),(73,794), 'box', 'EmergencyMeasures.RemovalOfmaterials'], 
        [(60,799),(73,811), 'box', 'EmergencyMeasures.Closing/InterdictionOfSupplyWells'],
        [(60,814),(73,827), 'box', 'EmergencyMeasures.BuildingsBan'], 
        [(60,831),(73,844), 'box', 'EmergencyMeasures.ExcavationBan'],
        [(60,849),(73,861), 'box', 'EmergencyMeasures.BanOnFoodConsumption'],
       

        
        [(513,759),(526,772), 'box', 'InstitutionalControlMeasures.SoilUse.RiskAssessment|InterventionPlan'],
        [(615,759),(627,772), 'box', 'InstitutionalControlMeasures.SoilUse.CommunicatedToEA'],
        [(690,757),(703,771), 'box', 'InstitutionalControlMeasures.SoilUse.Implemented'], 
        [(513,778),(526,790), 'box', 'InstitutionalControlMeasures.GroundwaterUse.RiskAssessment|InterventionPlan'],
        [(615,778),(627,790), 'box', 'InstitutionalControlMeasures.GroundwaterUse.CommunicatedToEA'],
        [(690,778),(703,790), 'box', 'InstitutionalControlMeasures.GroundwaterUse.Implemented'], 
        [(513,797),(526,810), 'box', 'InstitutionalControlMeasures.SurfaceWaterUse.RiskAssessment|InterventionPlan'],
        [(615,797),(627,810), 'box', 'InstitutionalControlMeasures.SurfaceWaterUse.CommunicatedToEA'], 
        [(690,796),(703,808), 'box', 'InstitutionalControlMeasures.SurfaceWaterUse.Implemented'],
        [(513,816),(526,828), 'box', 'InstitutionalControlMeasures.FoodConsumption.RiskAssessment|InterventionPlan'],
        [(615,816),(627,828), 'box', 'InstitutionalControlMeasures.FoodConsumption.CommunicatedToEA'],
        [(690,814),(703,826), 'box', 'InstitutionalControlMeasures.FoodConsumption.Implemented'], 
        [(513,832),(526,844), 'box', 'InstitutionalControlMeasures.BuildingsUse.RiskAssessment|InterventionPlan'],
        [(613,832),(626,844), 'box', 'InstitutionalControlMeasures.BuildingsUse.CommunicatedToEA'], 
        [(690,832),(703,844), 'box', 'InstitutionalControlMeasures.BuildingsUse.Implemented'],
       
        [(39,890),(52,902), 'box', 'RemediationMeasures.Pump&Treat'], 
        [(39,906),(52,918), 'box', 'RemediationMeasures.SoilVapourExtraction'], 
        [(39,919),(52,932), 'box', 'RemediationMeasures.AirSparging'], 
        [(39,935),(52,947), 'box', 'RemediationMeasures.Biosparging'], 
        [(39,953),(52,965), 'box', 'RemediationMeasures.Bioventing'],
        [(39,968),(52,981), 'box', 'RemediationMeasures.MultiphaseExtraction'],
        [(39, 986), (53, 1000), 'box', 'RemediationMeasures.ReductiveDichlorination'], 
        [(39,1005),(52,1018), 'box', 'RemediationMeasures.InSituHeatTreatment'],
        [(285,888),(298,901), 'box', 'RemediationMeasures.ChemicalOxidation'],
        [(285,905),(298,917), 'box', 'RemediationMeasures.ChemicalReduction'],
        [(285,918),(298,930), 'box', 'RemediationMeasures.ReactiveBarrier'],
        [(285,935),(298,947), 'box', 'RemediationMeasures.SoilWashing'],
        [(285,951),(298,963), 'box', 'RemediationMeasures.RemovalOfSoil/Residues'],
        [(288,966),(300,979), 'box', 'RemediationMeasures.FreePhaseRecovery'],
        [(288,984),(300,996), 'box', 'RemediationMeasures.GeotechnicalEncapsulation'],
        [(286,1002),(299,1014), 'box', 'RemediationMeasures.ContaminatedSoil/WasteCover'],
        [(533,889),(546,901), 'box', 'RemediationMeasures.PhysicalBarrier'],
        [(532,904),(545,917), 'box', 'RemediationMeasures.Hydraulicbarrier'], 
        [(532,922),(545,934), 'box', 'RemediationMeasures.Bioremediation'],
        [(532,938),(545,950), 'box', 'RemediationMeasures.Phytoremediation'],
        [(532,953),(545,966), 'box', 'RemediationMeasures.Biopile'], 
        [(532,971),(545,983), 'box', 'RemediationMeasures.MonitoredNaturalAttenuation'], 
        [(532,988),(545,1000), 'box', 'RemediationMeasures.Others'], 
        [(532,1004),(545,1017), 'box', 'RemediationMeasures.NoRemediationMeasures'],  

        [(39,1048),(52,1060), 'box', 'EngineeringControlMeasures.ProjectSuitability'],
        [(234,1050),(247,1062), 'box', 'EngineeringControlMeasures.Waterproofing'], 
        [(423,1050),(435,1062), 'box', 'EngineeringControlMeasures.Paving'],
        [(610,1049),(623,1061), 'box', 'EngineeringControlMeasures.Others']]

data_boxes = []
pixels_check= []
value = 0

img = cv2.imread(folder+'/'+forms_list[0])
img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
img = cv2.resize(img,(0,0),fx=1,fy=1)
#forms_list = ['CETESB_registry_2019-1.jpg', 'CETESB_registry_2019-10.jpg',
#              'CETESB_registry_2019-100.jpg', 'CETESB_registry_2019-1000.jpg', 'CETESB_registry_2019-1001.jpg']
for i,r in enumerate(roi):
    cv2.rectangle(img,(r[0][0],r[0][1]),(r[1][0],r[1][1]),(0,255,0),2)
    
cv2.imwrite(f'Detected_Textboxes_{year}_updated.jpg',img)
for j,y in enumerate(forms_list):
    data = []
    pixels = []
    img = cv2.imread(folder+"/"+ y)
    s_img = cv2.resize(img, (0, 0), fx=1,fy=1)
#     cv2.imwrite()
#     cv2.imshow(y,s_img)
#     cv2.waitKey(0)


    print(f'######### Extracting Data from {j} ##########')
    regex = re.compile(r'\d+')
    form_nb = regex.findall(y)[1]
    data.append(year)
    data.append(form_nb)
#     pixels.append(form_nb)
    for x,r in enumerate(roi):
        
        imgCrop = s_img[r[0][1]:r[1][1],r[0][0]:r[1][0]]
#         gray = cv2.cvtColor(imgCrop,cv2.COLOR_BGR2GRAY)

#         n_white_pix = np.sum(gray == 255)

#         pixels.append(n_white_pix)
        imgCrop_resized = cv2.resize(imgCrop, (112,112))

        X = image.img_to_array(imgCrop_resized)
        X = np.expand_dims(X,axis=0)
        images = np.vstack([X])
        val = model.predict(images)
        if val == 0:
            ### box is checked ###
            value = 1
            data.append(value)
        else: 
            ### box is not checked ###
            value = 0
            data.append(value)  
#     print(data)


#     pixels_check.append(pixels)
    data_boxes.append(data)

list_columns = [r[3] for r in (roi)]
list_columns.insert(0,'FormNb')
list_columns.insert(0,'Year')
# # Create the pandas DataFrame
# columns = ['FormNb', 'Activity.Industry', 'Activity.Commerce', 'Activity.Gas_Station', 'Activity.Residue', 'Activity.Accidents', 
#            'Activity.Agriculture', 'Activity.Unknown', 'Reuse', 'ManagementPhase.OccurenceAssessment', 
#            'ManagementPhase.LeakRemovalMeasures', 'ManagementPhase.ConfirmatoryInvestigation', 
#            'ManagementPhase.DetailedInvestigation&InterventionPlan', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring', 
#            'ManagementPhase.ClosingMonitoring', 'ManagementPhase.PreliminaryAssessment', 'ManagementPhase.ConfirmatoryInvestigation1',
#            'ManagementPhase.DetailedInvestigation', 'ManagementPhase.RiskAssessment', 'ManagementPhase.InterventionPlan', 
#            'ManagementPhase.RemediationProject', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring1', 
#            'ManagementPhase.ClosingMonitoring1', 'ContaminationSource.Storage', 'ContaminationSource.Disposal', 
#            'ContaminationSource.Production', 'ContaminationSource.Infiltration', 'ContaminationSource.Maintenance', 
#            'ContaminationSource.Accidents', 'ContaminationSource.AtmosphericEmissions', 'ContaminationSource.Unknown', 
#            'ContaminationSource.WastewaterTreatment', 'ImpactedMedia.SuperficialSoil.Inside', 'ImpactedMedia.SuperficialSoil.Outside', 
#            'ImpactedMedia.Soil.Inside', 'ImpactedMedia.Soil.Outside', 'ImpactedMedia.SurfaceWater.Inside',
#            'ImpactedMedia.SurfaceWater.Outside', 'ImpactedMedia.Groundwater.Inside', 'ImpactedMedia.Groundwater.Outside', 
#            'ImpactedMedia.Sediments.Inside', 'ImpactedMedia.Sediments.Outside', 'ImpactedMedia.Air.Inside', 
#            'ImpactedMedia.Air.Outside', 'ImpactedMedia.Biota.Inside', 'ImpactedMedia.Biota.Outside', 'Contaminants.Metals',
#            'Contaminants.OtherInorganic', 'Contaminants.HalogenatedSolvents', 'Contaminants.AromaticSolvents', 
#            'Contaminants.HalogenatedAromaticSolvents', 'Contaminants.PAHs', 'Contaminants.PCBs', 'Contaminants.Methane',
#            'Contaminants.AutomotiveFuels', 'Contaminants.OtherVapours/Gazes', 'Contaminants.Fenols', 'Contaminants.Biocides',
#            'Contaminants.Phthalates', 'Contaminants.Dioxins&Furans', 'Contaminants.Anilines', 'Contaminants.Radionuclides', 
#            'Contaminants.Microbiological', 'Contaminants.TPH', 'Contaminants.Others', 'Contaminants.FreePhase', 'Contaminants.POPs',
#            'EmergencyMeasures.IsolationOfTheArea', 'EmergencyMeasures.Ventilation/ExhaustionOfConfinedSpaces',
#            'EmergencyMeasures.MonitoringTheExplosivenessIndex', 'EmergencyMeasures.EnvironmentalMonitoring', 
#            'EmergencyMeasures.RemovalOfmaterials', 'EmergencyMeasures.Closing/InterdictionOfSupplyWells', 
#            'EmergencyMeasures.BuildingsBan', 'EmergencyMeasures.ExcavationBan', 'EmergencyMeasures.BanOnFoodConsumption',
#            'InstitutionalControlMeasures.SoilUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.SoilUse.CommunicatedToEA', 'InstitutionalControlMeasures.SoilUse.Implemented',
#            'InstitutionalControlMeasures.GroundwaterUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.GroundwaterUse.CommunicatedToEA', 'InstitutionalControlMeasures.GroundwaterUse.Implemented', 
#            'InstitutionalControlMeasures.SurfaceWaterUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.SurfaceWaterUse.CommunicatedToEA', 'InstitutionalControlMeasures.SurfaceWaterUse.Implemented',
#            'InstitutionalControlMeasures.FoodConsumption.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.FoodConsumption.CommunicatedToEA', 'InstitutionalControlMeasures.FoodConsumption.Implemented', 
#            'InstitutionalControlMeasures.BuildingsUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.BuildingsUse.CommunicatedToEA', 'InstitutionalControlMeasures.BuildingsUse.Implemented',
#            'RemediationMeasures.Pump&Treat', 'RemediationMeasures.SoilVapourExtraction', 'RemediationMeasures.AirSparging', 
#            'RemediationMeasures.Biosparging', 'RemediationMeasures.Bioventing', 'RemediationMeasures.MultiphaseExtraction', 
#            'RemediationMeasures.ReductiveDichlorination', 'RemediationMeasures.InSituHeatTreatment', 'RemediationMeasures.ChemicalOxidation', 'RemediationMeasures.ChemicalReduction', 'RemediationMeasures.ReactiveBarrier', 'RemediationMeasures.SoilWashing', 'RemediationMeasures.RemovalOfSoil/Residues', 'RemediationMeasures.FreePhaseRecovery', 'RemediationMeasures.GeotechnicalEncapsulation', 'RemediationMeasures.ContaminatedSoil/WasteCover', 'RemediationMeasures.PhysicalBarrier', 'RemediationMeasures.Hydraulicbarrier', 'RemediationMeasures.Bioremediation', 'RemediationMeasures.Phytoremediation', 'RemediationMeasures.Biopile', 'RemediationMeasures.MonitoredNaturalAttenuation', 'RemediationMeasures.Others', 'RemediationMeasures.NoRemediationMeasures', 'EngineeringControlMeasures.ProjectSuitability', 'EngineeringControlMeasures.Waterproofing', 
#            'EngineeringControlMeasures.Paving', 'EngineeringControlMeasures.Others']
# pixel_boxes = pd.DataFrame(pixels_check, columns=list_columns)
data_boxes = pd.DataFrame(data_boxes, columns=list_columns)

# pixel_boxes.to_excel(f'../../3.Output/dec2018/Pixel_box_trial_{year}_updated.xlsx')
data_boxes.to_excel(f'../Final_Boxes/{year}_model_classifier.xlsx')


######### Extracting Data from 0 ##########
1/1 [==============================] - 0s 74ms/step
######### Extracting Data from 1 ##########
1/1 [==============================] - 0s 152ms/step


1/1 [==============================] - 0s 67ms/step
######### Extracting Data from 2 ##########
1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 82ms/step


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize
